In [6]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import seaborn as sns
from keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten,Input
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import torch.nn as nn
import torch.nn.functional as F
from tensorflow.keras.optimizers import Adam
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

In [7]:
def num_flat_features(x):
    size = x.size()[1:]  # all dimensions except the batch dimension
    num_features = 1
    for s in size:
        num_features *= s
    return num_features

class VGG1(nn.Module):
    def __init__(self):
        super(VGG1, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1) # 200x200x3 -> 200x200x32
        self.flatten = nn.Flatten() # 100x100x32 --> 100*100*32
        self.linear1 = nn.Linear(32*100*100, 128)
        self.linear2 = nn.Linear(128, 1)
        
    def forward(self,x):
        x = self.conv1(x)
        x = F.max_pool2d(F.relu(x), 2) # 200x200x32 -> 100x100x32
        x = self.flatten(x)
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        return x

class VGG2(nn.Module):
    def __init__(self):
        super(VGG2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1) # 200x200x3 -> 200x200x32
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # 100x100x32 -> 100x100x64
        self.flatten = nn.Flatten() # 50x50x64 --> 50*50*64
        self.linear1 = nn.Linear(64*50*50, 128)
        self.linear2 = nn.Linear(128, 1)
        
    def forward(self,x):
        x = self.conv1(x)
        x = F.max_pool2d(F.relu(x), 2) # 200x200x32 -> 100x100x32
        x = self.conv2(x)
        x = F.max_pool2d(F.relu(x), 2) # 100x100x64 -> 50x50x64
        x = self.flatten(x)
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        return x
    
class VGG3(nn.Module):
    def __init__(self):
        super(VGG3, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1) # 200x200x3 -> 200x200x32
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # 100x100x32 -> 100x100x64
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1) # 50x50x64 -> 50x50x128
        self.flatten = nn.Flatten() # 25x25x128 --> 25*25*128
        self.linear1 = nn.Linear(128*25*25, 128)
        self.linear2 = nn.Linear(128, 1)
        
    def forward(self,x):
        x = self.conv1(x)
        x = F.max_pool2d(F.relu(x), 2) # 200x200x32 -> 100x100x32
        x = self.conv2(x)
        x = F.max_pool2d(F.relu(x), 2) # 100x100x64 -> 50x50x64
        x = self.conv3(x)
        x = F.max_pool2d(F.relu(x), 2) # 50x50x128 -> 25x25x128
        x = self.flatten(x)
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        return x


In [8]:
def train(model, train_loader, criterion, optimizer, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            
            outputs = model(images)
            loss = criterion(outputs, labels.float().unsqueeze(1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            predicted = torch.round(torch.sigmoid(outputs))
            total += labels.size(0)
            correct += (predicted == labels.float().unsqueeze(1)).sum().item()

    accuracy = (correct / total) * 100
    print(f"Accuracy on test set: {accuracy:.2f}%")
   
transform = transforms.Compose([transforms.Resize((200, 200)), transforms.ToTensor()]) 

train_dataset = datasets.ImageFolder('dataset_koala_vs_bear/train', transform=transform)
test_dataset = datasets.ImageFolder('dataset_koala_vs_bear/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=80, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=40, shuffle=False)





FileNotFoundError: Couldn't find any class folder in dataset_koala_vs_bear/train.

In [9]:
model = VGG1()
# Initialize the model, loss function, and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.002,momentum=0.7)
# Train the model
train(model, train_loader, criterion, optimizer)

# Test the model
test(model, test_loader)

NameError: name 'train_loader' is not defined

In [ ]:
model = VGG2()
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.005,momentum=0.7)
# Train the model
train(model, train_loader, criterion, optimizer)

# Test the model
test(model, test_loader)

: 

In [ ]:
model = VGG3()
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.009,momentum=0.7)
# Train the model
train(model, train_loader, criterion, optimizer)

# Test the model
test(model, test_loader)

Epoch 1/20, Loss: 0.6992658525705338
Epoch 2/20, Loss: 0.6902717322111129
Epoch 3/20, Loss: 0.6865376025438309
Epoch 4/20, Loss: 0.6830432116985321
Epoch 5/20, Loss: 0.6698315292596817
Epoch 6/20, Loss: 0.6353954076766968
Epoch 7/20, Loss: 0.6057696342468262
Epoch 8/20, Loss: 0.581680928170681
Epoch 9/20, Loss: 0.5588014185428619
Epoch 10/20, Loss: 0.5376873046159745
Epoch 11/20, Loss: 0.5579841151833534
Epoch 12/20, Loss: 0.48626005202531813
Epoch 13/20, Loss: 0.4453354313969612
Epoch 14/20, Loss: 0.5216520726680756
Epoch 15/20, Loss: 0.4181834921240807
Epoch 16/20, Loss: 0.41727293729782106
Epoch 17/20, Loss: 0.3773665361106396
Epoch 18/20, Loss: 0.32836082242429254
Epoch 19/20, Loss: 0.3986180517822504
Epoch 20/20, Loss: 0.27136526331305505
Accuracy on test set: 67.50%
